© Copyright 2020 Anthony D. Dutoi

This file is part of PyToon.

PyToon is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.

Consider a grid of square cells in 2D.  We will assume that the dimensions of these cells is much smaller than any length scale that we care about (taken to be infinitesemal eventually).  We will assert that the areas of these cells shall be constant even as they are subjected to rectangular distortions.  In particular, let there be a function $f$ of the horizontal coordinate $x$ which scales the cells in the y direction (so the horizontal boundaries of adjacent columns will no longer align).  In other words, for the $i^\text{th}$ column (and $j^\text{th}$ row), we have the following transformations (prime does not mean derivative)
$$
y_{ij}^\prime = f(x_i) \, y_{ij}
$$
and
$$
\Delta y_{i}^\prime = f(x_i) \, \Delta y_{i}
$$
So the corresponding transformations in the x direction must be
$$
\Delta x_i^\prime = \frac{1}{f(x_i)} \, \Delta x_i
$$
$$
x_i^\prime = \sum_{j=0}^{i} \Delta x_i = \sum_{j=0}^{i} \frac{1}{f(x_i)} \, \Delta x_i
$$
In the infinitesemal limit, we then obtain
$$
y^\prime = f(x) \, y
$$
and
$$
x^\prime = \int_0^x \text{d}x \frac{1}{f(x)}
$$

Now, assuming we want to describe a distortion which preserves the area in any given location (to mimic an incompressible fluid) but which gives a sinusoidal wave as a surface then we can use
$$
f(x) = 1-A\cos(2\pi\tilde{\nu}x) ~~~~; A<1~\text{is the amplitude of the wave (relative to the depth)}
$$
By plodding around with Mathematica and filling in the gaps by hand (easy to verify by straightforward differentiation), we have
$$
\int_0^x \text{d}x \frac{1}{1-A\cos(2\pi\tilde{\nu}x)} = \frac{1}{\pi\tilde{\nu}\sqrt{1-A^2}}\tan^{-1}\left(\sqrt{\frac{1+A}{1-A}}\tan(\pi\tilde{\nu}x)\right)
$$
This is sort of a nasty function which only works over the interval $\bar{\nu}x \in (-\frac{1}{2},+\frac{1}{2})$.  Even if we discount the unlikely event of landing exactly on one of the indeterminate endpoints, it will alias values outside this range to values inside this range, so the x coordinate needs to be broken up and handled piecewise.

More problematic is that, when used in the form above, we get a weird artifact at the cutting boundary which turns out not to be a numerical instability due to the divergent $\tan$ function (the combination with $\tan^{-1}$ is stable).  The problem is that
$$
\lim_{\tilde{\nu}x\rightarrow\frac{1}{2}^-} \left[\tilde{\nu}x^\prime = \frac{1}{\pi\sqrt{1-A^2}}\tan^{-1}\left(\sqrt{\frac{1+A}{1-A}}\tan(\pi\tilde{\nu}x)\right)\right] = \frac{1}{2\sqrt{1-A^2}}
$$
where the superscript "$-$" means "limit from the left".  One can intuit that the mapping should return its own argument at $0$ and $\pm\frac{1}{2}$, so it seems to be off by a factor of $(1-A^2)^{-1/2}$.  So I removed it and it looks really good; not sure what went wrong, maybe working to fast and missed some detail or extra condition?.

In [1]:
from math import pi, sin, cos, tan, atan, sqrt, modf
import random
from util import int_round
from pytoon import struct, composite, circle, polygon, positional_transform

rand = lambda: random.uniform(-0.35, 0.35)

def modf_min(x):
    f, i = modf(x)
    if f<-0.5:
        return f+1, i-1
    elif f>=0.5:
        return f-1, i+1
    else:
        return f, i

In [2]:
def distort(L, T, t):
    A = 0.1
    def f(p):
        x, y = p        
        xP = x/L - t/T
        y = y * (1 - A*cos(2*pi*xP))
        xPf, xPi = modf_min(xP)
        xP = xPi + atan(sqrt((1+A)/(1-A)) * tan(pi*xPf)) / pi
        x = L * (xP + t/T)
        return x,y
    return f, 1

In [3]:
X, Y, d = 600, 200, 15

Nx, Ny = int_round(X/d), int_round(Y/d)
Px, Py = [i*X/Nx for i in range(Nx+1)], [i*Y/Ny for i in range(Ny+1)]

positions = []
for x in Px:
    for y in Py:
        positions += [(x+rand()*d, y+rand()*d)]
special = ((Nx+1)*(Ny+1))//2 - (Ny+1)//4 - 1

block              = polygon(lstyle=False, fstyle="#202020")
molecule           = circle(radius=4, lstyle=("black",0.25), fstyle="#3bba9c")
molecules          = [molecule(center=(x,y)) for x,y in positions]
surface            = [(i*X/125,Y*1.05) for i in range(125+1)]

wavy = [i*X/125 for i in range(63)]
wavy = [(x,10) for x in wavy] + [(x,-10) for x in reversed(wavy)]
wavy = polygon(points=wavy, lstyle=("black",0.25), fstyle="green").S(0.40).R(60).T(0,-15)

image = composite([
    composite([
        polygon(points=[(0,-20),*surface,(X,-20)], lstyle=False, fstyle="#233142"),
        *molecules,
        wavy.S(1).T(60,0), wavy.S(1.1).T(150,0), wavy.S(1.2).T(195,0), wavy.S(0.7).T(300,0), wavy.S(1.5).T(470,0), wavy.S(1).T(510,0), wavy.S(0.8).T(530,0)
    ], transform=positional_transform(distort, L=X/2, T="T", t="_t_").animated(Dt=0.1)),
    molecules[special](radius=2, fstyle="red"),
    block(points=[(-10,20),(-10,-15),(X+10,-15),(X+10,20)], fstyle="brown").T(0,-30),
    block(points=[(-15,-46),(-15,Y*1.5),(20,Y*1.5),(20,-46)]),
    block(points=[(X-20,-46),(X-20,Y*1.5),(X+15,Y*1.5),(X+15,-46)])
])

In [4]:
T = 3

image(T=T).svg("waterwave", title="Wave on the Surface of Water", time=(0,T), duration=T, background="#202020", controls=False)

[link to animation file](files/waterwave.svg)